In [15]:
import pandas as pd
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

from config import mypass

from scapy import *

In [16]:
museum_directory_csv_file = "museums.csv"
museum_directory_df = pd.read_csv(museum_directory_csv_file, encoding = "UTF-8")
museum_directory_df.head()

/Users/melanieeppright/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9,14,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Museum ID,Museum Name,Legal Name,Alternate Name,Museum Type,Institution Name,Street Address (Administrative Location),City (Administrative Location),State (Administrative Location),Zip Code (Administrative Location),...,Latitude,Longitude,Locale Code (NCES),County Code (FIPS),State Code (FIPS),Region Code (AAM),Employer ID Number,Tax Period,Income,Revenue
0,8400200098,ALASKA AVIATION HERITAGE MUSEUM,ALASKA AVIATION HERITAGE MUSEUM,NaN,HISTORY MUSEUM,NaN,4721 AIRCRAFT DR,ANCHORAGE,AK,99502,...,61.17925,-149.97254,1.0,20.0,2.0,6,920071852,201312.0,602912.0,550236.0
1,8400200117,ALASKA BOTANICAL GARDEN,ALASKA BOTANICAL GARDEN INC,NaN,"ARBORETUM, BOTANICAL GARDEN, OR NATURE CENTER",NaN,4601 CAMPBELL AIRSTRIP RD,ANCHORAGE,AK,99507,...,61.16890,-149.76708,4.0,20.0,2.0,6,920115504,201312.0,1379576.0,1323742.0
2,8400200153,ALASKA CHALLENGER CENTER FOR SPACE SCIENCE TEC...,ALASKA CHALLENGER CENTER FOR SPACE SCIENCE TEC...,NaN,SCIENCE & TECHNOLOGY MUSEUM OR PLANETARIUM,NaN,9711 KENAI SPUR HWY,KENAI,AK,99611,...,60.56149,-151.21598,3.0,122.0,2.0,6,921761906,201312.0,740030.0,729080.0
3,8400200143,ALASKA EDUCATORS HISTORICAL SOCIETY,ALASKA EDUCATORS HISTORICAL SOCIETY,NaN,HISTORIC PRESERVATION,NaN,214 BIRCH STREET,KENAI,AK,99611,...,60.56280,-151.26597,3.0,122.0,2.0,6,920165178,201412.0,0.0,0.0
4,8400200027,ALASKA HERITAGE MUSEUM,ALASKA AVIATION HERITAGE MUSEUM,NaN,HISTORY MUSEUM,NaN,301 W NORTHERN LIGHTS BLVD,ANCHORAGE,AK,99503,...,61.17925,-149.97254,1.0,20.0,2.0,6,920071852,201312.0,602912.0,550236.0


In [55]:
clean_museum_dir = museum_directory_df[['Museum ID','Museum Name', 'State (Administrative Location)']].copy()
renamed_museum_dir = clean_museum_dir.rename(columns={'Museum ID': 'Museum_ID',
                                                        'Museum Name': 'Museum_Name', 
                                                       'State (Administrative Location)': 'State'})
indexed_renamed_museum_dir = renamed_museum_dir.set_index("Museum_Name")
Dropped_Duplicates_df = indexed_renamed_museum_dir.drop_duplicates()
Dropped_Duplicates_df.head()

,Museum_ID,State
Museum_Name,,
ALASKA AVIATION HERITAGE MUSEUM,8400200098,AK
ALASKA BOTANICAL GARDEN,8400200117,AK
ALASKA CHALLENGER CENTER FOR SPACE SCIENCE TECHNOLOGY,8400200153,AK
ALASKA EDUCATORS HISTORICAL SOCIETY,8400200143,AK
ALASKA HERITAGE MUSEUM,8400200027,AK


In [39]:
connection_string = f"root:{mypass}@127.0.0.1/Museums_db"
engine = create_engine(f'mysql://{connection_string}')

In [40]:
Museum_json_file = "Museum_Reviews/traveler_type_USonly.json"
Museum_Reviews_df = pd.read_json(Museum_json_file)
Museum_Reviews_df.head()

,Gettysburg Heritage Center,Hudson River Museum,The Aurora Ice Museum,Chrysler Museum of Art,Pink Palace Museum,Gettysburg Seminary Ridge Museum,Oklahoma Railway Museum,Ships of the Sea Maritime Museum,LBJ Presidential Library,Museum Mile,...,New England Air Museum,GulfQuest National Maritime Museum of the Gulf of Mexico,Mohave Museum of History and Arts,Ellis Island,San Diego Model Railroad Museum,Superstition Mountain Museum,Please Touch Museum,McClung Museum of Natural History and Culture,NSU Art Museum,Morikami Museum & Japanese Gardens
0,88,25,132,133,163,84,23,73,399,0,...,73,45,14,658,100,46,567,48,21,195
1,86,22,204,155,84,170,8,237,605,8,...,27,46,35,729,71,76,37,20,70,365
2,17,2,35,52,12,34,1,24,144,1,...,20,8,6,119,23,17,14,10,20,64
3,2,4,7,17,7,4,1,14,107,0,...,9,3,3,16,4,2,13,2,6,8
4,33,15,106,93,41,64,4,82,302,4,...,7,23,11,367,28,48,53,19,48,265


In [60]:
Review_Averages_df = Museum_Reviews_df.mean(axis=0)
Reset_Review_Averages_df = Review_Averages.reset_index(level=0)
Renamed_Averages_df = Reset_Review_Averages_df.rename(columns={'index': 'Museum_Name', 0:'Rating'})
Indexed_Museums = Renamed_Averages_df.set_index("Museum_Name")
Indexed_Museums.head()

,Rating
Museum_Name,
Gettysburg Heritage Center,45.2
Hudson River Museum,13.6
The Aurora Ice Museum,96.8
Chrysler Museum of Art,90.0
Pink Palace Museum,61.4


In [35]:
engine.table_names()

['Museum_Directory', 'Museum_Reviews']

In [57]:
Dropped_Duplicates_df.to_sql(name='Museum_Directory', con=engine, if_exists='append', index=True)

In [58]:
pd.read_sql_query('select * from Museum_Directory', con=engine).head()

,Museum_Name,Museum_ID,State
0,ALASKA AVIATION HERITAGE MUSEUM,8400200098,AK
1,ALASKA BOTANICAL GARDEN,8400200117,AK
2,ALASKA CHALLENGER CENTER FOR SPACE SCIENCE TEC...,8400200153,AK
3,ALASKA EDUCATORS HISTORICAL SOCIETY,8400200143,AK
4,ALASKA HERITAGE MUSEUM,8400200027,AK


In [61]:
Indexed_Museums.to_sql(name='Museum_Reviews', con=engine, if_exists='append', index=True)

In [62]:
pd.read_sql_query('select * from Museum_Reviews', con=engine).head()

,Museum_Name,Rating
0,1940 Air Terminal Museum,17.4
1,21c Museum Hotel,31.2
2,45th Infantry Division Museum,32.0
3,Abbe Museum,39.0
4,Abby Aldrich Rockefeller Folk Art Museum,78.6
